In [9]:
import pandas as pd
import pickle
from helper_functions import Helper
from helper_functions import Coinmetrics

In [10]:
# TODO: taxonomy?

In [ ]:
if __name__ == "__main__":
    # set args
    CW_IN_FP = '../data/derived/cm_to_coinapi_cw.pkl'
    ASSET_IN_FP = '../data/clean/asset_universe_dict.pickle'
    CM_API_FP = '../../admin/coinmetrics.txt'
    BASE_URL = 'https://api.coinmetrics.io/v4/'
    PANEL_OUT_FP = '../data/raw/coinmetrics_panel_hourly.pkl'
    MACRO_OUT_FP = '../data/raw/coinmetrics_macro_hourly.pkl'
    TARGET_US_EXCHANGES = ['binance.us',  'bitstamp', 'coinbase', 'crypto.com', 'ftx.us', 
        'gemini', 'kraken', 'kucoin']
    TARGET_EXCHANGE_METRICS = ['open_interest_reported_future_usd', 'volume_reported_future_usd_1h', 'volume_reported_spot_usd_1h']
    TARGET_FREQ = '1h'
    STUDY_START = '2016-07-01'
    STUDY_END = '2023-01-03'
    TARGET_METRICS = ['AdrActCnt', 'AdrActContCnt', 'AdrActRecCnt', 'AdrActSentCnt', 'AdrBalCnt',
        'AdrBalUSD100Cnt', 'AdrBalUSD100KCnt', 'AdrBalUSD10Cnt', 'AdrBalUSD10KCnt', 'AdrBalUSD10MCnt',
        'AdrBalUSD1Cnt', 'AdrBalUSD1KCnt', 'AdrBalUSD1MCnt', 'CapAct1yrUSD',
        'CapFutExp10yrUSD', 'CapMVRVCur', 'CapMVRVFF', 'CapMrktCurUSD', 'CapMrktEstUSD', 'CapMrktFFUSD',
        'CapRealUSD', 'ContERC1155Cnt', 'ContERC20Cnt', 'ContERC721Cnt',
        'DiffMean', 'FeeMeanUSD', 'FeeMedUSD', 'FeeRevPct', 'FeeTotUSD', 'FlowInExInclUSD', 'FlowInExUSD',
        'FlowMinerIn0HopAllUSD', 'FlowMinerIn1HopAllUSD', 'FlowMinerNet0HopAllUSD', 'FlowMinerNet1HopAllUSD',
        'FlowMinerOut0HopAllUSD', 'FlowMinerOut1HopAllUSD', 'FlowOutExInclUSD', 'FlowOutExUSD', 'FlowTfrFromExCnt',
        'FlowTfrFromExInclCnt', 'FlowTfrToExCnt', 'FlowTfrToExInclCnt', 'FlowToCLCont', 'GasUsedTx', 'GasUsedTxMean',
        'HashRate', 'IssContNtv', 'IssContPctAnn', 'IssContPctDay', 'IssContUSD', 'IssTotUSD', 'MCRC', 'MCTC',
        'MOMR', 'MRI0HopAll30d', 'MRI1HopAll30d', 'NDF', 'NVTAdj', 'NVTAdjFF', 'PriceUSD', 'PuellMulCont', 'PuellMulRev',
        'PuellMulTot', 'RCTC', 'RVTAdj', 'ReferenceRateUSD', 'RevAllTimeUSD', 'RevHash1yAvgNtv', 'RevHash1yAvgUSD',
        'RevHashUSD', 'RevUSD', 'SER', 'SOPR', 'SOPROut', 'SenderCntCLCont', 'SenderTotCLCont', 'SplyAct10yr',
        'SplyAct180d', 'SplyAct1d', 'SplyAct1yr', 'SplyAct2yr', 'SplyAct30d', 'SplyAct3yr', 'SplyAct4yr', 'SplyAct5yr',
        'SplyAct7d', 'SplyAct90d', 'SplyActEver', 'SplyActPct1yr', 'SplyAdrBalUSD1', 'SplyAdrBalUSD10',
        'SplyAdrBalUSD100', 'SplyAdrBalUSD100K', 'SplyAdrBalUSD10K', 'SplyAdrBalUSD10M', 'SplyAdrBalUSD1K',
        'SplyAdrBalUSD1M', 'SplyAdrTop100', 'SplyAdrTop10Pct', 'SplyAdrTop1Pct', 'SplyBurntUSD', 'SplyContUSD',
        'SplyCur', 'SplyExUSD', 'SplyExpFut10yr', 'SplyFF', 'SplyMiner0HopAllUSD', 'SplyMiner1HopAllUSD',
        'SplyRvv180d', 'SplyRvv1yr', 'SplyRvv2yr', 'SplyRvv30d', 'SplyRvv3yr', 'SplyRvv4yr', 'SplyRvv5yr', 
        'SplyRvv7d', 'SplyRvv90d', 'SplyUTXOLoss', 'SplyUTXOProf', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD',    
        'TxTfrValContCallUSD', 'TxTfrValDayDst', 'TxTfrValDayDstMean', 'TxTfrValMeanUSD', 'TxTfrValMedUSD',
        'TxTfrValNtv', 'TxTfrValRecNtv', 'TxTfrValSentNtv', 'TxTfrValUSD', 'UTXOAgeMean', 'UTXOAgeMed', 
        'UTXOAgeValMean', 'UTXOCnt', 'UTXODay', 'UTXOLossCnt', 'UTXOLossUnrealUSD', 'UTXOProfCnt',
        'UTXOProfUnrealUSD', 'VelAct1yr', 'VelCur1yr', 'WalActCnt', 'WalActRecCnt', 'WalActSentCnt', 'WalBalCnt',
        'mempool_count', 'mempool_fee', 'mempool_fee_mean', 'mempool_fee_median', 'mempool_feerate_mean',  
        'mempool_feerate_median', 'mempool_output_value', 'mempool_size', 'mempool_vsize', 
        'mining_reward_mean', 'mining_reward_spread']
    
    # import api key
    with open(CM_API_FP) as f:
        API_KEY = f.readlines()
        API_KEY = API_KEY[0].strip()
    BASE_PARAMS = {'api_key': API_KEY}

    # Import asset universe and cw
    cw_df = pd.read_pickle(CW_IN_FP)
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe = Helper.findUniqueAssets(asset_universe_dict)

    # Pull info on assets
    assets_df = Coinmetrics.pullAssetInfo(BASE_URL, BASE_PARAMS, asset_universe+['usdt', 'usdc'])

    # pull meta data on target exchanges
    exchanges_df = Coinmetrics.pullExchangeInfo(BASE_URL, BASE_PARAMS, TARGET_US_EXCHANGES)

    # pull meta data markets and subset down to target markets
    markets_df = Coinmetrics.pullAndFormRelevantMarkets(exchanges_df, assets_df, BASE_URL, BASE_PARAMS, asset_universe)

    # pull reference rates for USDC and USDT to use for exchange rate calculations
    usd_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END, 
                                          TARGET_FREQ, ['usdc', 'usdt'], ['ReferenceRateUSD'])
    usd_df = Coinmetrics.cleanUSDPrices(usd_df, TARGET_FREQ)

    # pull prices and volumes for all markets of interest
    ohlcv_df = Coinmetrics.pullOHLCV(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, TARGET_FREQ, markets_df)

    # pull asset metrics
    panel_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END,
                                            TARGET_FREQ, asset_universe, TARGET_METRICS)
    panel_df = Coinmetrics.cleanPanel(panel_df)

    # pull exchange metrics
    ex_metrics_df = Coinmetrics.pullExchangeMetrics(BASE_URL, BASE_PARAMS, TARGET_EXCHANGE_METRICS, 
                                                    STUDY_START, STUDY_END, TARGET_FREQ)
    ex_metrics_df = Coinmetrics.cleanExchangeMetrics(ex_metrics_df, TARGET_EXCHANGE_METRICS, TARGET_US_EXCHANGES)

    # form macro dataframe
    macro_df = usd_df.merge(ex_metrics_df, on=['date'], how='outer', validate='one_to_one')
    macro_df.to_pickle(MACRO_OUT_FP)

In [19]:
# pull bid ask prices
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)

The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 0.51 seconds.
The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 1.26 seconds.
The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 0.88 seconds.
The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 0.92 seconds.
The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 0.56 seconds.
The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443): Read timed out. (read timeout=5)
Retrying after 1.19 seconds.
The API call failed with error: HTTPSConnectionPool(host='api.coinmetrics.io', port=443)

KeyboardInterrupt: 

In [ ]:
# form the panel
prices_df = Coinmetrics.cleanMarketPanels(ohlcv_df, TARGET_FREQ, markets_df, usd_df, ba_df)
panel_df = panel_df.merge(prices_df, on=['date', 'asset'], how='outer', validate='one_to_one')
panel_df.to_pickle(PANEL_OUT_FP)